---
title: Exploring BiG-SLICE query result
teaching: 0
exercises: 20
questions:
    - " How do I annotate BiG-SLICE query result"
objectives:
    - "Enrich BiG-FAM hits with other BGCflow results"
keypoints:
    - "Different BGCflow outputs can be combined to enrich BiG-SLICE query network"
---

In this episode, we will explore BiG-SLICE query hits of _S. venezuelae_ genomes with the [BiG-FAM database (version 1.0.0, run 6)](https://bigfam.bioinformatics.nl/home). You can download [the `.ipynb` file of this episode](https://github.com/NBChub/bgcflow_tutorial/blob/gh-pages/_episodes/06-bigslice_query-part2.ipynb) and run it from your own directory.

### Table of Contents
1. [BGCflow Paths Configuration](#1)
2. [Raw BiG-SLICE query hits](#2)
3. [A glimpse of the data distribution](#3)
    - [Sanity Check: How many gene clusters predicted by antiSMASH?](#3.1)
    - [How many BGCs can be assigned to BiG-FAM gene cluster families?](#3.2)
    - [A closer look to BiG-FAM distributions](#3.3)
4. [Annotate Network with information from BiG-SCAPE and GTDB](#4)
5. [Import the annotation to Cytoscape](#5)

### Libraries & Functions

In [ ]:
# load libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from pathlib import Path
import json

In [ ]:
# generate data
def gcf_hits(df_gcf, cutoff=900):
    """
    Filter bigslice result based on distance threshold to model and generate data.
    """
    mask = df_gcf.loc[:, "membership_value"] <= cutoff
    df_gcf_filtered = df_gcf[mask]
    bgcs = df_gcf_filtered.bgc_id.unique()
    gcfs = df_gcf_filtered.gcf_id.unique()
    print(
    f"""BiG-SLICE query with BiG-FAM run 6, distance cutoff {cutoff}
Number of bgc hits : {len(bgcs)}/{len(df_gcf.bgc_id.unique())}
Number of GCF hits : {len(gcfs)}""")
    return df_gcf_filtered

# visualization
def plot_overview(data):
    """
    Plot BGC hits distribution from BiG-SLICE Query
    """
    ranks = data.gcf_id.value_counts().index
    sns.set_theme()
    sns.set_context("paper")
    fig, axes = plt.subplots(1, 2, figsize=(25, 20))
    plt.figure(figsize = (25,25))

    # first plot
    sns.histplot(data=data, x='membership_value', 
                 kde=True, ax=axes[0],)

    # second plot
    sns.boxplot(data=data, y='gcf_id', x='membership_value', 
                orient='h', ax=axes[1], order=ranks)

    # Add in points to show each observation
    sns.stripplot(x="membership_value", y="gcf_id", data=data,
                  jitter=True, size=3, linewidth=0, color=".3", 
                  ax=axes[1], orient='h', order=ranks)
    return

## BGCflow Paths Configuration <a name="1"></a>
Customize the cell below to your BGCflow result paths

In [ ]:
# interim data
bigslice_query = Path("/datadrive/home/matinnu/bgcflow_data/interim/bigslice/query/s_venezuelae_antismash_6.0.1/")

# processed data
bigslice_query_processed = Path("/datadrive/bgcflow/data/processed/s_venezuelae/bigslice/query_as_6.0.1/")
bigscape_result = Path("/datadrive/home/matinnu/bgcflow_data/processed/s_venezuelae/bigscape/for_cytoscape_antismash_6.0.1/2022-06-21 02_46_35_df_clusters_0.30.csv")
gtdb_table = Path("/datadrive/home/matinnu/bgcflow_data/processed/s_venezuelae/tables/df_gtdb_meta.csv")

# output path
output_path = Path("../tables/bigslice")
output_path.mkdir(parents=True, exist_ok=True)

## Raw BiG-SLICE query hits <a name="2"></a>
First, let's see the raw data from BiG-SLICE query hits. We have extracted individual tables from the SQL database.

In [ ]:
! tree /datadrive/bgcflow/data/interim/bigslice/query/s_venezuelae_antismash_6.0.1/

We will first look at these two tables:
- `bgc` table or the file `bgc.csv`
- `gcf_membership` table or the file `gcf_membership.csv`

The data from the interim folder has been processed for downstream analysis in the processed directory:

In [ ]:
! tree /datadrive/bgcflow/data/processed/s_venezuelae/bigslice/query_as_6.0.1/

In [ ]:
# load the two tables
df_bgc = pd.read_csv(bigslice_query / "bgc.csv")
# BGC table from bigslice
df_bgc.loc[:, 'genome_id'] = [Path(i).name for i in df_bgc.orig_folder] # will be put in the main bgcflow code
# GCF membership table from bigslice
df_gcf_membership = pd.read_csv(bigslice_query / "gcf_membership.csv")

The `gcf_membership` table lists the top 10 closest BiG-FAM GCF models (order shown as `rank` column) and the euclidean distance to the model (`membership_value`). Smaller `membership_value` means that our BGC has a closer or more similar features with the models. Note that we are querying against run 6 in the BiG-FAM model, with threshold of 900, Therefore, if the `membership_value` is above 900, it is less likely that our BGC belongs to that gcf model.

In [ ]:
df_gcf_membership.head()

The `bgc_id` column in `gcf_membership` table refers to the `id` column in `bgc` table. Therefore, we can enrich our hits with the metadata contained in `bgc` table

In [ ]:
df_bgc.head()

## A glimpse of the data distribution <a name="3"></a>
### Sanity Check: How many gene clusters predicted by antiSMASH? <a name="3.1"></a>

In [ ]:
print(f"There are {len(df_bgc)} BGCs predicted from {len(df_bgc.orig_folder.unique())} genomes.")

### How many BGCs can be assigned to BiG-FAM gene cluster families? <a name="3.2"></a>
BiG-SLICE calculate the feature distance of a BGC to BiG-FAM models (which is a centroid of each Gene Cluster Families generated from 1.2 million BGCs). Though it's not that accurate, it can give us a glimpse of the distribution of our BGCs within the database.

In [ ]:
sns.set_theme()
sns.set_context("paper")
sns.histplot(df_gcf_membership, x='membership_value', kde=True)
for c in [900, 1200, 1500]:
    gcf_hits(df_gcf_membership, c)

Depending on the distance cutoffs, we can assign our BGCs to a different numbers of GCF model. The default cutoffs is 900 (run 6). In our data, 382 out of 515 BGCs can be assigned to 114 BiG-FAM GCF. Do note that the number of assigned GCF can be smaller if we only consider the first hit (the query returns 10 hits).

Smaller number means a closer distance to the GCF model. For further analysis, we will stick with the default cutoff.

### A closer look to BiG-FAM distributions <a name="3.3"></a>
BGCflow already cleans the data for downstream processing. The processed bigslice query can be found in `bgcflow/data/processed/s_venezuelae/bigslice/query_as_6.0.1/`.

In [ ]:
data = pd.read_csv(bigslice_query_processed / "query_network.csv")
plot_overview(data)

On the figure above, we can see the distance distribution of our query to the model, and how each models have varying degree of distances. But, this data includes the top 10 hits, so 1 BGCs can be assigned to multiple GCFs.

Let's see again only for the first hit.

In [ ]:
n_hits = 1 # get only the first hit
n_hits_only = data.loc[:, "rank"].isin(np.arange(n_hits))
data_1 = data[n_hits_only]
print(f"For the top {n_hits} hit, {len(data_1.bgc_id.unique())} BGCs can be mapped to {len(data_1.gcf_id.unique())} GCF")
plot_overview(data_1)

## Annotate Network with information from BiG-SCAPE and GTDB <a name="4"></a>
These network will only be meaningful when we enrich it with metadata. We can use information from our BiG-SCAPE runs, taxonomic information from GTDB-tk, and other tables generated by BGCflow. 

In [ ]:
# Enrich with BiG-SCAPE
df_annotation = pd.read_csv(bigscape_result, index_col=0)
df_annotation.loc[:, "bigslice_query"] = "query"
for i in data["gcf_id"].unique():
    df_annotation.loc[i, "bigslice_query"] = "model"

# enrich with GTDB
df_gtdb = pd.read_csv(gtdb_table).set_index("genome_id")
for i in df_annotation.index:
    genome_id = df_annotation.loc[i, "genome_id"]
    if type(genome_id) == str:
        for item in ["gtdb_release", "Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species", "Organism"]:
            df_annotation.loc[i, item] = df_gtdb.loc[genome_id, item]

# enrich with bgc info - will be put in the main bgcflow code
bgc_info = df_bgc.copy()
bgc_info["bgc_id"] = [str(i).replace(".gbk", "") for i in bgc_info["orig_filename"]]
bgc_info = bgc_info.set_index("bgc_id")
for i in df_annotation.index:
    try:
        df_annotation.loc[i, "on_contig_edge"] = bgc_info.loc[i, "on_contig_edge"]
        df_annotation.loc[i, "length_nt"] = bgc_info.loc[i, "length_nt"]
    except KeyError:
        pass

In [ ]:
df_annotation.head()

In [ ]:
df_annotation.to_csv("../tables/bigslice/enriched_query_annotation.csv")

## Import the annotation to Cytoscape <a name="5"></a>
Download the `enriched_query_annotation.csv` and import it to enrich the nodes in Cytoscape network.
Using the new annotations, play around and explore the network to find interesting BGCs and their BiG-FAM models.